In [ ]:
# 이 코드 예시는 기존 파이프라인 구조를 확장하여 다양한 Shape(원, 사다리꼴, 스파이크, 기둥, 직사각형, 타원, 언덕, 안개, 다각형, 파형, 실제 음원)과
# Pattern(선형, 랜덤, n번 반복 후 t초 정지, 볼록, 임의 함수 기반)을 추가하는 방법의 개략적 구조를 제안한다.
# 각 Shape와 Pattern은 DB 기반 스펙트로그램에 노이즈 패턴을 추가할 수 있도록 구성.
# 실제 구현에서는 각 Shape/Pattern의 마스킹 로직, 수학적 변환, 오디오 합성 등이 필요하다.
#
# PEP8 스타일 준수 및 한글 주석
# 주어진 Shape와 Pattern들을 유연하게 팩토리 및 파이프라인에 통합한 예시 코드.

import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
from abc import ABC, abstractmethod


class SpectrogramModifier:
    def __init__(self, sample_rate=16000, n_fft=1024, hop_length=512,
                 noise_strength=0.1, noise_type='normal', noise_params=None):
        self.sample_rate = sample_rate
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.noise_strength = noise_strength
        self.noise_type = noise_type
        self.noise_params = noise_params if noise_params else {}
        self.signal = None
        self.signal_with_noise = None
        self.S_db = None

    def _generate_normal_noise(self, length, params):
        mean = params.get('mean', 0.0)
        std = params.get('std', 1.0)
        return np.random.normal(mean, std, length)

    def _generate_uniform_noise(self, length, params):
        low = params.get('low', -1.0)
        high = params.get('high', 1.0)
        return np.random.uniform(low, high, length)

    def _generate_perlin_noise(self, length, params):
        def fade(t):
            return 6*t**5 - 15*t**4 + 10*t**3

        seed = params.get('seed', 42)
        np.random.seed(seed)
        perm = np.arange(256)
        np.random.shuffle(perm)
        perm = np.stack([perm, perm]).flatten()
        scale = params.get('scale', 50.0)
        xs = np.linspace(0, length/scale, length)
        xi = np.floor(xs).astype(int)
        xf = xs - xi
        xi = xi % 256
        left_hash = perm[xi]
        right_hash = perm[xi+1]
        u = fade(xf)
        left_grad = ((left_hash & 1)*2-1)*xf
        right_grad = ((right_hash & 1)*2-1)*(xf-1)
        noise = (1-u)*left_grad + u*right_grad
        noise = noise / np.max(np.abs(noise))
        return noise

    def generate_noise(self, signal):
        length = len(signal)
        nt = self.noise_type
        p = self.noise_params
        if nt == 'normal':
            noise = self._generate_normal_noise(length, p)
        elif nt == 'uniform':
            noise = self._generate_uniform_noise(length, p)
        elif nt == 'perlin':
            noise = self._generate_perlin_noise(length, p)
        else:
            noise = np.zeros_like(signal)
        return signal + noise * self.noise_strength

    def compute_spectrogram(self, signal):
        self.signal = signal
        self.signal_with_noise = self.generate_noise(signal)
        S = np.abs(librosa.stft(
            self.signal_with_noise, n_fft=self.n_fft, hop_length=self.hop_length, window='hann'))
        self.S_db = librosa.amplitude_to_db(S, ref=np.max)
        return self.S_db

    def _get_freqs(self):
        return np.linspace(0, self.sample_rate / 2, self.S_db.shape[0])

    def _get_times(self):
        return librosa.frames_to_time(np.arange(self.S_db.shape[1]),
                                      sr=self.sample_rate,
                                      hop_length=self.hop_length)

    def apply_dB_mask(self, dB_mask):
        self.S_db += dB_mask

    def plot_spectrogram(self, show_labels=True, colormap='magma', title='Spectrogram'):
        if self.S_db is None:
            raise ValueError("compute_spectrogram() 먼저 호출")
        fig, ax = plt.subplots(figsize=(12, 6))
        img = librosa.display.specshow(
            self.S_db,
            sr=self.sample_rate,
            hop_length=self.hop_length,
            x_axis='time',
            y_axis='linear',
            ax=ax,
            cmap=colormap
        )
        if show_labels:
            ax.set_xlabel('Time (s)')
            ax.set_ylabel('Frequency (Hz)')
            ax.set_title(title)
            fig.colorbar(img, ax=ax, format="%+2.0f dB")
        else:
            ax.axis('off')
            plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
        plt.tight_layout(pad=0.5)
        return fig, ax


import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
from abc import ABC, abstractmethod

class Distribution(ABC):
    @abstractmethod
    def sample(self, shape):
        pass

class NormalDistribution(Distribution):
    def __init__(self, mean, stddev):
        self.mean = mean
        self.stddev = stddev
    def sample(self, shape):
        return np.random.normal(self.mean, self.stddev, shape)

class UniformDistribution(Distribution):
    def __init__(self, min_val, max_val):
        self.min_val = min_val
        self.max_val = max_val
    def sample(self, shape):
        return np.random.uniform(self.min_val, self.max_val, shape)

class NoneDistribution(Distribution):
    def sample(self, shape):
        return np.zeros(shape)

class PoissonDistribution(Distribution):
    def __init__(self, lam=5):
        self.lam = lam
    def sample(self, shape):
        # Poisson은 양의 정수형이라 평균을 lam으로 정규화
        # dB 값으로 사용하려면 변환 필요할 수 있으나 여기서는 그대로 사용
        return np.random.poisson(self.lam, shape).astype(float)

class ExponentialDistribution(Distribution):
    def __init__(self, scale=1.0):
        self.scale = scale
    def sample(self, shape):
        # 지수분포는 양수에 치우쳐 있으므로 평균 0 만들기 위해 shift
        noise = np.random.exponential(self.scale, shape)
        noise -= np.mean(noise)
        return noise

class CauchyDistribution(Distribution):
    def sample(self, shape):
        noise = np.random.standard_cauchy(shape)
        # 극단값 제거
        noise = np.clip(noise, -10, 10)
        noise = noise / np.max(np.abs(noise))
        return noise

class LaplaceDistribution(Distribution):
    def __init__(self, loc=0.0, scale=1.0):
        self.loc = loc
        self.scale = scale
    def sample(self, shape):
        return np.random.laplace(self.loc, self.scale, shape)

class BetaDistribution(Distribution):
    def __init__(self, a=2.0, b=2.0):
        self.a = a
        self.b = b
    def sample(self, shape):
        noise = np.random.beta(self.a, self.b, shape)
        # [0,1] -> [-1,1]
        noise = (noise - 0.5)*2
        return noise

class GammaDistribution(Distribution):
    def __init__(self, shape_param=2.0, scale=1.0):
        self.shape_param = shape_param
        self.scale = scale
    def sample(self, shape):
        noise = np.random.gamma(self.shape_param, self.scale, shape)
        # Gamma는 양수만 존재, 평균0 만들고 정규화
        noise -= np.mean(noise)
        noise = noise / (np.std(noise) + 1e-9)
        return noise

class ChiSquareDistribution(Distribution):
    def __init__(self, df=2):
        self.df = df
    def sample(self, shape):
        noise = np.random.chisquare(self.df, shape)
        noise -= np.mean(noise)
        noise = noise / (np.std(noise) + 1e-9)
        return noise

class DistributionEngine:
    def create(self, dist_type, **params):
        dist_type = dist_type.lower()
        if dist_type == "normal":
            return NormalDistribution(params.get('mean', 0), params.get('stddev', 1))
        elif dist_type == "uniform":
            return UniformDistribution(params.get('min', -1), params.get('max', 1))
        elif dist_type == "poisson":
            return PoissonDistribution(params.get('lam', 5))
        elif dist_type == "exponential":
            return ExponentialDistribution(params.get('scale', 1.0))
        elif dist_type == "cauchy":
            return CauchyDistribution()
        elif dist_type == "laplace":
            return LaplaceDistribution(params.get('loc', 0.0), params.get('scale', 1.0))
        elif dist_type == "beta":
            return BetaDistribution(params.get('a', 2.0), params.get('b', 2.0))
        elif dist_type == "gamma":
            return GammaDistribution(params.get('shape_param', 2.0), params.get('scale', 1.0))
        elif dist_type == "chisquare":
            return ChiSquareDistribution(params.get('df', 2))
        else:
            return NoneDistribution()


##################################
# 다양한 Shape 클래스 구현 (DBShape)
# 여기서는 마스킹 로직을 간략히 표기
##################################

class DBShape(ABC):
    @abstractmethod
    def create_mask(self, spectro_shape, spectro_mod):
        pass

class BaseShape(DBShape):
    def __init__(self, distribution_engine, distribution_type='none', distribution_params=None):
        self.distribution_engine = distribution_engine
        self.distribution_type = distribution_type
        self.distribution_params = distribution_params if distribution_params else {}

    def _sample_distribution(self, shape):
        dist_obj = self.distribution_engine.create(self.distribution_type, **self.distribution_params)
        return dist_obj.sample(shape)

    @abstractmethod
    def generate_shape_mask(self, spectro_shape, spectro_mod):
        pass

    def create_mask(self, spectro_shape, spectro_mod):
        shape_mask = self.generate_shape_mask(spectro_shape, spectro_mod)
        # shape_mask는 도형 내부 1, 외부 0
        # distribution_type이 'none'이 아니라면 distribution 샘플링
        if self.distribution_type != 'none':
            dist_values = self._sample_distribution(spectro_shape)
            return shape_mask * dist_values
        else:
            # distribution을 사용하지 않는 경우, shape_mask 자체에 일정한 dB값을 더하도록
            # 이 경우 shape_mask 자체가 dB값을 담고 있을 수도 있음.
            return shape_mask    

# 원 형태 노이즈
class CircleDBShape(BaseShape):
    def __init__(self, center_freq, center_time, radius_freq, radius_time, strength_dB,
                 distribution_engine, distribution_type='none', distribution_params=None):
        super().__init__(distribution_engine, distribution_type, distribution_params)
        self.center_freq = center_freq
        self.center_time = center_time
        self.radius_freq = radius_freq
        self.radius_time = radius_time
        self.strength_dB = strength_dB

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        freqs = spectro_mod._get_freqs()
        times = spectro_mod._get_times()
        ff, tt = np.meshgrid(freqs, times, indexing='ij')
        dist = ((ff - self.center_freq)**2 / (self.radius_freq**2) +
                (tt - self.center_time)**2 / (self.radius_time**2))
        circle = (dist <= 1).astype(float)
        # distribution_type='none'일 때 사용될 기본 dB 값
        return circle * self.strength_dB
    
# 사다리꼴 형태 노이즈
class TrapezoidDBShape(BaseShape):
    def __init__(self, freq_min, freq_max, time_min, time_max, slope_freq, slope_time, strength_dB,
                 distribution_engine, distribution_type='none', distribution_params=None):
        super().__init__(distribution_engine, distribution_type, distribution_params)
        self.freq_min = freq_min
        self.freq_max = freq_max
        self.time_min = time_min
        self.time_max = time_max
        self.slope_freq = slope_freq
        self.slope_time = slope_time
        self.strength_dB = strength_dB

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        mask = np.zeros(spectro_shape)
        freqs = spectro_mod._get_freqs()
        times = spectro_mod._get_times()
        freq_mask = (freqs >= self.freq_min) & (freqs <= self.freq_max)
        time_mask = (times >= self.time_min) & (times <= self.time_max)
        f_inds = np.where(freq_mask)[0]
        t_inds = np.where(time_mask)[0]
        if len(f_inds) == 0 or len(t_inds) == 0:
            return mask
        f_dist = (freqs[f_inds] - self.freq_min) / (self.freq_max - self.freq_min)
        t_dist = (times[t_inds] - self.time_min) / (self.time_max - self.time_min)
        for i, fi in enumerate(f_inds):
            for j, ti in enumerate(t_inds):
                val = self.strength_dB * (1 - abs(f_dist[i]-0.5)*2*self.slope_freq) * (1 - abs(t_dist[j]-0.5)*2*self.slope_time)
                mask[fi, ti] += val
        return mask

# 스파이크 형태 (주파수나 시간 특정 지점에 뾰족한 형태)
class SpikeDBShape(BaseShape):
    def __init__(self, distribution_engine, center_freq, center_time, radius_freq, radius_time, strength_dB,
                 rotate=(0, 0), distribution_type='none', distribution_params=None):
        super().__init__(distribution_engine, distribution_type, distribution_params)
        self.center_freq = center_freq
        self.center_time = center_time
        self.radius_freq = radius_freq
        self.radius_time = radius_time
        self.strength_dB = strength_dB
        self.rotate_deg = rotate[0]

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        freqs = spectro_mod._get_freqs()
        times = spectro_mod._get_times()
        ff, tt = np.meshgrid(freqs, times, indexing='ij')

        f_shift = ff - self.center_freq
        t_shift = tt - self.center_time

        angle_rad = np.deg2rad(self.rotate_deg)
        cos_a = np.cos(angle_rad)
        sin_a = np.sin(angle_rad)

        f_rot = f_shift * cos_a - t_shift * sin_a
        t_rot = f_shift * sin_a + t_shift * cos_a

        dist = np.sqrt((f_rot**2) / (self.radius_freq**2) + (t_rot**2) / (self.radius_time**2))
        spike = np.exp(-dist*5) * self.strength_dB
        return spike

# 기둥 형태
class PillarDBShape(BaseShape):
    def __init__(self, freq_min, freq_max, strength_dB, distribution_engine, distribution_type='none', distribution_params=None):
        super().__init__(distribution_engine, distribution_type, distribution_params)
        self.freq_min = freq_min
        self.freq_max = freq_max
        self.strength_dB = strength_dB

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        mask = np.zeros(spectro_shape)
        freqs = spectro_mod._get_freqs()
        freq_indices = np.where((freqs >= self.freq_min) & (freqs <= self.freq_max))[0]
        if len(freq_indices) == 0:
            return mask
        mask[freq_indices, :] += self.strength_dB
        return mask

# 직사각형 형태
class RectangleDBShape(BaseShape):
    def __init__(self, freq_min, freq_max, time_min, time_max, strength_dB,
                 distribution_engine, distribution_type='none', distribution_params=None):
        super().__init__(distribution_engine, distribution_type, distribution_params)
        self.freq_min = freq_min
        self.freq_max = freq_max
        self.time_min = time_min
        self.time_max = time_max
        self.strength_dB = strength_dB

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        mask = np.zeros(spectro_shape)
        freqs = spectro_mod._get_freqs()
        times = spectro_mod._get_times()
        freq_indices = np.where((freqs >= self.freq_min) & (freqs <= self.freq_max))[0]
        time_indices = np.where((times >= self.time_min) & (times <= self.time_max))[0]
        if len(freq_indices) == 0 or len(time_indices) == 0:
            return mask
        mask[np.ix_(freq_indices, time_indices)] += self.strength_dB
        return mask

# 타원 형태
class EllipseDBShape(BaseShape):
    def __init__(self, center_freq, center_time, radius_freq, radius_time, strength_dB,
                 distribution_engine, distribution_type='none', distribution_params=None):
        super().__init__(distribution_engine, distribution_type, distribution_params)
        self.center_freq = center_freq
        self.center_time = center_time
        self.radius_freq = radius_freq
        self.radius_time = radius_time
        self.strength_dB = strength_dB

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        freqs = spectro_mod._get_freqs()
        times = spectro_mod._get_times()
        ff, tt = np.meshgrid(freqs, times, indexing='ij')
        dist = ((ff - self.center_freq)**2 / (self.radius_freq**2) +
                (tt - self.center_time)**2 / (self.radius_time**2))
        ellipse = (dist <= 1).astype(float)
        return ellipse * self.strength_dB

# 언덕 형태
class HillDBShape(BaseShape):
    def __init__(self, freq_center, time_center, freq_width, time_width, strength_dB,
                 distribution_engine, distribution_type='none', distribution_params=None):
        super().__init__(distribution_engine, distribution_type, distribution_params)
        self.freq_center = freq_center
        self.time_center = time_center
        self.freq_width = freq_width
        self.time_width = time_width
        self.strength_dB = strength_dB

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        freqs = spectro_mod._get_freqs()
        times = spectro_mod._get_times()
        ff, tt = np.meshgrid(freqs, times, indexing='ij')
        dist = np.sqrt(((ff - self.freq_center)**2) / (self.freq_width**2) +
                       ((tt - self.time_center)**2) / (self.time_width**2))
        hill = (1 - dist)
        hill[hill < 0] = 0
        return hill * self.strength_dB
    
# 안개(fog): 전체 구간에 랜덤 저강도 노이즈
class FogDBShape(BaseShape):
    def __init__(self, strength_dB, coverage=1.0, distribution_engine=None, distribution_type='none', distribution_params=None):
        super().__init__(distribution_engine, distribution_type, distribution_params)
        self.strength_dB = strength_dB
        self.coverage = coverage

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        random_map = np.random.uniform(0, 1, spectro_shape)
        fog = (random_map < self.coverage).astype(float) * self.strength_dB * (np.random.randn(*spectro_shape) * 0.1)
        return fog

# 다각형(polygon): 주파수-시간 좌표계 상에 다각형을 정의하고 내부를 채움
class PolygonDBShape(BaseShape):
    def __init__(self, vertices, strength_dB, distribution_engine=None, distribution_type='none', distribution_params=None):
        super().__init__(distribution_engine, distribution_type, distribution_params)
        self.vertices = vertices
        self.strength_dB = strength_dB

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        from matplotlib.path import Path
        freqs = spectro_mod._get_freqs()
        times = spectro_mod._get_times()
        ff, tt = np.meshgrid(freqs, times, indexing='ij')

        poly_points = [(v[0], v[1]) for v in self.vertices]
        path = Path(poly_points)
        points = np.vstack((ff.ravel(), tt.ravel())).T
        inside = path.contains_points(points).reshape(ff.shape)
        mask = np.zeros(spectro_shape)
        mask[inside] = self.strength_dB
        return mask

# 파형(wave pattern): sin, cos 형태로 dB 변조
class WavePatternDBShape(BaseShape):
    def __init__(self, axis='time', frequency=1.0, strength_dB=5.0, distribution_engine=None, distribution_type='none', distribution_params=None):
        super().__init__(distribution_engine, distribution_type, distribution_params)
        self.axis = axis
        self.frequency = frequency
        self.strength_dB = strength_dB

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        freqs = spectro_mod._get_freqs()
        times = spectro_mod._get_times()
        mask = np.zeros(spectro_shape)
        if self.axis == 'time':
            wave = np.sin(2 * np.pi * self.frequency * times)
            mask += wave[np.newaxis, :] * self.strength_dB
        else:
            wave = np.sin(2 * np.pi * self.frequency * freqs)
            mask += wave[:, np.newaxis] * self.strength_dB
        return mask

# 실제 음원 삽입: 외부 wav/mp3 파일의 스펙트럼을 특정 구간에 합성
class RealWorldNoiseDBShape(BaseShape):
    def __init__(self, audio_path, freq_min, freq_max, time_min, time_max, strength_dB=0,
                 distribution_engine=None, distribution_type='none', distribution_params=None):
        super().__init__(distribution_engine, distribution_type, distribution_params)
        self.audio_path = audio_path
        self.freq_min = freq_min
        self.freq_max = freq_max
        self.time_min = time_min
        self.time_max = time_max
        self.strength_dB = strength_dB

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        # 음원 로드 후 스펙트럼 처리
        y, sr = librosa.load(self.audio_path, sr=spectro_mod.sample_rate)
        S = np.abs(librosa.stft(y, n_fft=spectro_mod.n_fft, hop_length=spectro_mod.hop_length))
        S_db = librosa.amplitude_to_db(S, ref=np.max)

        freqs = spectro_mod._get_freqs()
        times = spectro_mod._get_times()
        freq_indices = np.where((freqs >= self.freq_min) & (freqs <= self.freq_max))[0]
        time_indices = np.where((times >= self.time_min) & (times <= self.time_max))[0]

        if len(freq_indices) == 0 or len(time_indices) == 0:
            return np.zeros(spectro_shape)

        f_cut = min(len(freq_indices), S_db.shape[0])
        t_cut = min(len(time_indices), S_db.shape[1])

        mask = np.zeros(spectro_shape)
        mask[freq_indices[:f_cut], time_indices[:t_cut]] += S_db[:f_cut, :t_cut] + self.strength_dB
        return mask


##################################
# 패턴 클래스 구현 (시간/주파수 축으로 linear, random, n-time-linear-t-sleep, convex, function)
##################################

class Pattern(ABC):
    @abstractmethod
    def create_mask(self, spectro_shape, spectro_mod, shape_factory, distribution_engine):
        pass

# 선형 패턴: 주어진 shape를 time 또는 freq 방향으로 일정 간격 반복
class LinearPattern(Pattern):
    def __init__(self, shape_name, shape_params, direction='time', repeat=5, spacing=1.0):
        self.shape_name = shape_name
        self.shape_params = shape_params
        self.direction = direction
        self.repeat = repeat
        self.spacing = spacing

    def create_mask(self, spectro_shape, spectro_mod, shape_factory, distribution_engine):
        mask = np.zeros(spectro_shape)
        # direction에 따라 shape를 일정 간격으로 반복
        for i in range(self.repeat):
            offset = i * self.spacing
            # shape_params 수정
            # 예: time_offset 적용
            params = self.shape_params.copy()
            if 'center_time' in params:
                params['center_time'] = params['center_time'] + offset if self.direction == 'time' else params['center_time']
            if 'center_freq' in params:
                params['center_freq'] = params['center_freq'] + offset if self.direction == 'freq' else params['center_freq']
            shape_obj = shape_factory.create(self.shape_name, **params)
            if shape_obj is not None:
                mask += shape_obj.create_mask(spectro_shape, spectro_mod)
        return mask

# 랜덤 패턴: 일정 구간 내 shape를 랜덤 위치로 n개 생성
class RandomPattern(Pattern):
    def __init__(self, shape_name, shape_params, n=10, freq_range=None, time_range=None):
        self.shape_name = shape_name
        self.shape_params = shape_params
        self.n = n
        self.freq_range = freq_range if freq_range else (0, 8000)
        self.time_range = time_range if time_range else (0, 10)

    def create_mask(self, spectro_shape, spectro_mod, shape_factory, distribution_engine):
        mask = np.zeros(spectro_shape)
        for _ in range(self.n):
            params = self.shape_params.copy()
            if 'center_freq' in params:
                params['center_freq'] = np.random.uniform(*self.freq_range)
            if 'center_time' in params:
                params['center_time'] = np.random.uniform(*self.time_range)
            shape_obj = shape_factory.create(self.shape_name, **params)
            if shape_obj is not None:
                mask += shape_obj.create_mask(spectro_shape, spectro_mod)
        return mask

# n번 linear 후 t초 쉬기 (n_linear_repeat_t_time_sleep) 패턴 예시
class NLinearRepeatTSleepPattern(Pattern):
    def __init__(self, shape_name, shape_params, repeat=3, repeat_time=0.5, sleep_time=5.0, start_time=0.0):
        self.shape_name = shape_name
        self.shape_params = shape_params
        self.repeat = repeat
        self.repeat_time = repeat_time
        self.sleep_time = sleep_time
        self.start_time = start_time

    def create_mask(self, spectro_shape, spectro_mod, shape_factory, distribution_engine):
        mask = np.zeros(spectro_shape)
        current_time = self.start_time
        for _ in range(self.repeat):
            params = self.shape_params.copy()
            if 'center_time' in params:
                params['center_time'] = current_time
            shape_obj = shape_factory.create(self.shape_name, **params)
            if shape_obj is not None:
                mask += shape_obj.create_mask(spectro_shape, spectro_mod)
            current_time += self.repeat_time
        # sleep_time동안 다음 패턴 없음 (단순히 gap)
        current_time += self.sleep_time
        return mask

# 볼록 패턴(convex): 주어진 구간에 convex 형태로 shape 배치(간략 구현)
class ConvexPattern(Pattern):
    def __init__(self, shape_name, shape_params, freq_min, freq_max, time_min, time_max, n=10):
        self.shape_name = shape_name
        self.shape_params = shape_params
        self.freq_min = freq_min
        self.freq_max = freq_max
        self.time_min = time_min
        self.time_max = time_max
        self.n = n

    def create_mask(self, spectro_shape, spectro_mod, shape_factory, distribution_engine):
        mask = np.zeros(spectro_shape)
        # freq, time 값을 convex 형태로 분포 (중앙 밀집)
        freqs = np.linspace(self.freq_min, self.freq_max, self.n)
        times = np.linspace(self.time_min, self.time_max, self.n)
        # 중간이 제일 촘촘하게 분포하도록 가중(간단히 Gaussian 분포 샘플링 가능)
        for i in range(self.n):
            params = self.shape_params.copy()
            if 'center_freq' in params:
                params['center_freq'] = freqs[i]
            if 'center_time' in params:
                params['center_time'] = times[i]
            shape_obj = shape_factory.create(self.shape_name, **params)
            if shape_obj is not None:
                mask += shape_obj.create_mask(spectro_shape, spectro_mod)
        return mask

class HorizontalLineDBShape(BaseShape):
    def __init__(self, center_freq, strength_dB, thickness=1,
                 distribution_engine=None, distribution_type='none', distribution_params=None):
        super().__init__(distribution_engine, distribution_type, distribution_params)
        self.center_freq = center_freq
        self.strength_dB = strength_dB
        self.thickness = thickness

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        mask = np.zeros(spectro_shape)
        freqs = spectro_mod._get_freqs()
        freq_idx = np.argmin(np.abs(freqs - self.center_freq))
        start_idx = max(freq_idx - self.thickness//2, 0)
        end_idx = min(freq_idx + self.thickness//2 + 1, spectro_shape[0])
        mask[start_idx:end_idx, :] += self.strength_dB
        return mask

class VerticalLineDBShape(BaseShape):
    def __init__(self, center_time, strength_dB, thickness=1,
                 distribution_engine=None, distribution_type='none', distribution_params=None):
        super().__init__(distribution_engine, distribution_type, distribution_params)
        self.center_time = center_time
        self.strength_dB = strength_dB
        self.thickness = thickness

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        mask = np.zeros(spectro_shape)
        times = spectro_mod._get_times()
        time_idx = np.argmin(np.abs(times - self.center_time))
        start_idx = max(time_idx - self.thickness//2, 0)
        end_idx = min(time_idx + self.thickness//2 + 1, spectro_shape[1])
        mask[:, start_idx:end_idx] += self.strength_dB
        return mask

class HorizontalRangeDistributionDBShape(DBShape):
    # 주파수 구간에 대해 분포 기반 dB 값 추가
    def __init__(self, freq_min, freq_max, strength_dB, distribution='gaussian', distribution_params=None):
        self.freq_min = freq_min
        self.freq_max = freq_max
        self.strength_dB = strength_dB
        self.distribution = distribution
        self.distribution_params = distribution_params if distribution_params else {}

    def _get_distribution(self, values, distribution, params):
        if distribution == 'gaussian':
            sigma = params.get('sigma', 5)
            center = (values.min() + values.max()) / 2
            dist = np.exp(-0.5 * ((values - center) / sigma)**2)
        else:
            # 기본 gaussian 사용
            sigma = params.get('sigma', 5)
            center = (values.min() + values.max()) / 2
            dist = np.exp(-0.5 * ((values - center) / sigma)**2)
        dist /= dist.max()
        return dist

    def create_mask(self, spectro_shape, spectro_mod):
        mask = np.zeros(spectro_shape)
        freqs = spectro_mod._get_freqs()
        freq_indices = np.where((freqs >= self.freq_min) & (freqs <= self.freq_max))[0]
        if len(freq_indices) == 0:
            return mask
        dist_values = freqs[freq_indices]
        energy_distribution = self._get_distribution(dist_values, self.distribution, self.distribution_params)
        # freq_indices 범위에 strength_dB * distribution 적용
        # 모든 시간축에 동일 적용
        for i, fi in enumerate(freq_indices):
            mask[fi, :] += self.strength_dB * energy_distribution[i]
        return mask

class VerticalRangeDistributionDBShape(DBShape):
    # 시간 구간에 분포 기반 dB 값 추가
    def __init__(self, time_min, time_max, strength_dB, distribution='gaussian', distribution_params=None):
        self.time_min = time_min
        self.time_max = time_max
        self.strength_dB = strength_dB
        self.distribution = distribution
        self.distribution_params = distribution_params if distribution_params else {}

    def _get_distribution(self, values, distribution, params):
        if distribution == 'gaussian':
            sigma = params.get('sigma', 0.5)
            center = (values.min() + values.max()) / 2
            dist = np.exp(-0.5 * ((values - center) / sigma)**2)
        else:
            # 기본 gaussian
            sigma = params.get('sigma', 0.5)
            center = (values.min() + values.max()) / 2
            dist = np.exp(-0.5 * ((values - center) / sigma)**2)
        dist /= dist.max()
        return dist

    def create_mask(self, spectro_shape, spectro_mod):
        mask = np.zeros(spectro_shape)
        times = spectro_mod._get_times()
        time_indices = np.where((times >= self.time_min) & (times <= self.time_max))[0]
        if len(time_indices) == 0:
            return mask
        dist_values = times[time_indices]
        energy_distribution = self._get_distribution(dist_values, self.distribution, self.distribution_params)
        for i, ti in enumerate(time_indices):
            mask[:, ti] += self.strength_dB * energy_distribution[i]
        return mask

# 임의 함수 기반 패턴: 사용자가 freq/time -> dB 변환 함수 제공
class FunctionPattern(Pattern):
    def __init__(self, func):
        # func: callable(freqs, times) -> dB_mask
        self.func = func

    def create_mask(self, spectro_shape, spectro_mod, shape_factory, distribution_engine):
        freqs = spectro_mod._get_freqs()
        times = spectro_mod._get_times()
        ff, tt = np.meshgrid(freqs, times, indexing='ij')
        mask = self.func(ff, tt)
        return mask


##################################
# 팩토리 구현
##################################

class ShapeFactory:
    def __init__(self, distribution_engine):
        self.distribution_engine = distribution_engine

    def create(self, shape_name, **kwargs):
        if shape_name == "circle":
            return CircleDBShape(
                distribution_engine=self.distribution_engine,
                center_freq=kwargs['center_freq'],
                center_time=kwargs['center_time'],
                radius_freq=kwargs['radius_freq'],
                radius_time=kwargs['radius_time'],
                strength_dB=kwargs['strength_dB'],
                distribution_type=kwargs.get('distribution_type', 'none'),
                distribution_params=kwargs.get('distribution_params', None)
            )
        elif shape_name == "trapezoid":
            return TrapezoidDBShape(
                distribution_engine=self.distribution_engine,
                freq_min=kwargs['freq_min'],
                freq_max=kwargs['freq_max'],
                time_min=kwargs['time_min'],
                time_max=kwargs['time_max'],
                slope_freq=kwargs['slope_freq'],
                slope_time=kwargs['slope_time'],
                strength_dB=kwargs['strength_dB'],
                distribution_type=kwargs.get('distribution_type', 'none'),
                distribution_params=kwargs.get('distribution_params', None)
            )
        elif shape_name == "rectangle":
            return RectangleDBShape(
                distribution_engine=self.distribution_engine,
                freq_min=kwargs['freq_min'],
                freq_max=kwargs['freq_max'],
                time_min=kwargs['time_min'],
                time_max=kwargs['time_max'],
                strength_dB=kwargs['strength_dB'],
                distribution_type=kwargs.get('distribution_type', 'none'),
                distribution_params=kwargs.get('distribution_params', None)
            )
        elif shape_name == "ellipse":
            return EllipseDBShape(
                distribution_engine=self.distribution_engine,
                center_freq=kwargs['center_freq'],
                center_time=kwargs['center_time'],
                radius_freq=kwargs['radius_freq'],
                radius_time=kwargs['radius_time'],
                strength_dB=kwargs['strength_dB'],
                distribution_type=kwargs.get('distribution_type', 'none'),
                distribution_params=kwargs.get('distribution_params', None)
            )
        elif shape_name == "fog":
            return FogDBShape(
                distribution_engine=self.distribution_engine,
                strength_dB=kwargs['strength_dB'],
                coverage=kwargs.get('coverage', 1.0),
                distribution_type=kwargs.get('distribution_type', 'none'),
                distribution_params=kwargs.get('distribution_params', None)
            )
        elif shape_name == "polygon":
            return PolygonDBShape(
                distribution_engine=self.distribution_engine,
                vertices=kwargs['vertices'],
                strength_dB=kwargs['strength_dB'],
                distribution_type=kwargs.get('distribution_type', 'none'),
                distribution_params=kwargs.get('distribution_params', None)
            )
        elif shape_name == "wave_pattern":
            return WavePatternDBShape(
                distribution_engine=self.distribution_engine,
                axis=kwargs.get('axis', 'time'),
                frequency=kwargs['frequency'],
                strength_dB=kwargs['strength_dB'],
                distribution_type=kwargs.get('distribution_type', 'none'),
                distribution_params=kwargs.get('distribution_params', None)
            )
        elif shape_name == "real_world_noise":
            return RealWorldNoiseDBShape(
                distribution_engine=self.distribution_engine,
                audio_path=kwargs['audio_path'],
                freq_min=kwargs['freq_min'],
                freq_max=kwargs['freq_max'],
                time_min=kwargs['time_min'],
                time_max=kwargs['time_max'],
                strength_dB=kwargs['strength_dB'],
                distribution_type=kwargs.get('distribution_type', 'none'),
                distribution_params=kwargs.get('distribution_params', None)
            )
        elif shape_name == "spike":
            return SpikeDBShape(
                distribution_engine=self.distribution_engine,
                center_freq=kwargs['center_freq'],
                center_time=kwargs['center_time'],
                radius_freq=kwargs['radius_freq'],
                radius_time=kwargs['radius_time'],
                strength_dB=kwargs['strength_dB'],
                rotate=kwargs.get('rotate', (0, 0)),
                distribution_type=kwargs.get('distribution_type', 'none'),
                distribution_params=kwargs.get('distribution_params', None)
            )
        elif shape_name == "hill":
            return HillDBShape(
                distribution_engine=self.distribution_engine,
                freq_center=kwargs['freq_center'],
                time_center=kwargs['time_center'],
                freq_width=kwargs['freq_width'],
                time_width=kwargs['time_width'],
                strength_dB=kwargs['strength_dB'],
                distribution_type=kwargs.get('distribution_type', 'none'),
                distribution_params=kwargs.get('distribution_params', None)
            )
        elif shape_name == "pillar":
            return PillarDBShape(
                distribution_engine=self.distribution_engine,
                freq_min=kwargs['freq_min'],
                freq_max=kwargs['freq_max'],
                strength_dB=kwargs['strength_dB'],
                distribution_type=kwargs.get('distribution_type', 'none'),
                distribution_params=kwargs.get('distribution_params', None)
            )
        elif shape_name == "horizontal_line":
            return HorizontalLineDBShape(
                distribution_engine=self.distribution_engine,
                center_freq=kwargs['center_freq'],
                strength_dB=kwargs['strength_dB'],
                thickness=kwargs.get('thickness', 1),
                distribution_type=kwargs.get('distribution_type', 'none'),
                distribution_params=kwargs.get('distribution_params', None)
            )
        elif shape_name == "vertical_line":
            return VerticalLineDBShape(
                distribution_engine=self.distribution_engine,
                center_time=kwargs['center_time'],
                strength_dB=kwargs['strength_dB'],
                thickness=kwargs.get('thickness', 1),
                distribution_type=kwargs.get('distribution_type', 'none'),
                distribution_params=kwargs.get('distribution_params', None)
            )
        elif shape_name == "horizontal_range_dist_db":
            return HorizontalRangeDistributionDBShape(
                kwargs['freq_min'], kwargs['freq_max'],
                kwargs['strength_dB'],
                kwargs.get('distribution', 'gaussian'),
                kwargs.get('distribution_params', {})
                )
        elif shape_name == "vertical_range_dist_db":
            return VerticalRangeDistributionDBShape(
                kwargs['time_min'], kwargs['time_max'],
                kwargs['strength_dB'],
                kwargs.get('distribution', 'gaussian'),
                kwargs.get('distribution_params', {})
                )
        else:
            raise ValueError(f"Unknown shape name: {shape_name}")

class PatternFactory:
    def create(self, pattern_name, params):
        if pattern_name == "linear":
            return LinearPattern(params['shape_name'], params['shape_params'],
                                 params.get('direction', 'time'),
                                 params.get('repeat', 5), params.get('spacing', 1.0))
        elif pattern_name == "random":
            return RandomPattern(params['shape_name'], params['shape_params'],
                                 params.get('n', 10),
                                 params.get('freq_range', (0, 8000)),
                                 params.get('time_range', (0, 10)))
        elif pattern_name == "n_linear_repeat_t_time_sleep":
            return NLinearRepeatTSleepPattern(params['shape_name'], params['shape_params'],
                                              params.get('repeat', 3),
                                              params.get('repeat_time', 0.5),
                                              params.get('sleep_time', 5.0),
                                              params.get('start_time', 0.0))
        elif pattern_name == "convex":
            return ConvexPattern(params['shape_name'], params['shape_params'],
                                 params['freq_min'], params['freq_max'],
                                 params['time_min'], params['time_max'],
                                 params.get('n', 10))
        elif pattern_name == "function":
            return FunctionPattern(params['func'])
        else:
            return None


class NoisePipeline:
    def __init__(self, spectro_mod):
        self.spectro_mod = spectro_mod
        self.distribution_engine = DistributionEngine()
        self.shape_factory = ShapeFactory(self.distribution_engine)  # 여기서 인자 전달 가능
        self.pattern_factory = PatternFactory()
        self.shapes = []
        self.patterns = []

    def add_shape(self, shape):
        self.shapes.append(shape)
        return self

    def add_pattern(self, pattern):
        if pattern is not None:
            self.patterns.append(pattern)
        return self

    def generate(self, signal):
        spec = self.spectro_mod.compute_spectrogram(signal)
        total_mask = np.zeros_like(spec)
        for shape in self.shapes:
            shape_mask = shape.create_mask(spec.shape, self.spectro_mod)
            total_mask += shape_mask
        for pattern in self.patterns:
            pattern_mask = pattern.create_mask(spec.shape, self.spectro_mod, self.shape_factory, self.distribution_engine)
            total_mask += pattern_mask
        self.spectro_mod.apply_dB_mask(total_mask)
        return self.spectro_mod.S_db



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
from abc import ABC, abstractmethod
import random
import json

# BaseShape 클래스 정의
class BaseShape(ABC):
    def __init__(self, distribution_engine=None, distribution_type='none', distribution_params=None):
        self.distribution_engine = distribution_engine
        self.distribution_type = distribution_type
        self.distribution_params = distribution_params

    @abstractmethod
    def generate_shape_mask(self, spectro_shape, spectro_mod):
        pass

    def create_mask(self, spectro_shape, spectro_mod):
        shape_mask = self.generate_shape_mask(spectro_shape, spectro_mod)
        # distribution_type에 따라 shape_mask 조정
        if self.distribution_type != 'none' and self.distribution_engine:
            if self.distribution_type == 'normal':
                mask_distribution = self.distribution_engine.normal(shape_mask.shape, 
                                                                      self.distribution_params.get('mean', 1), 
                                                                      self.distribution_params.get('std', 1))
                shape_mask *= mask_distribution
            # 다른 distribution_type에 대한 처리 추가 가능
        return shape_mask

    def plot_mask(self, mask):
        plt.figure(figsize=(10, 4))
        librosa.display.specshow(mask, sr=16000, hop_length=512, x_axis='time', y_axis='linear', cmap='cool')
        plt.colorbar(format="%+2.0f dB")
        plt.title(f"{self.__class__.__name__} Mask")
        plt.show()

# HorizontalLineDBShape 클래스 정의
class HorizontalLineDBShape(BaseShape):
    def __init__(self, center_freq, strength_dB, thickness=1,
                 distribution_engine=None, distribution_type='none', distribution_params=None):
        super().__init__(distribution_engine, distribution_type, distribution_params)
        self.center_freq = center_freq
        self.strength_dB = strength_dB
        self.thickness = int(thickness)  # thickness를 정수로 변환

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        mask = np.zeros(spectro_shape)
        freqs = spectro_mod._get_freqs()
        freq_idx = np.argmin(np.abs(freqs - self.center_freq))
        start_idx = max(freq_idx - self.thickness // 2, 0)
        end_idx = min(freq_idx + self.thickness // 2 + 1, spectro_shape[0])
        mask[start_idx:end_idx, :] += self.strength_dB
        return mask

# VerticalLineDBShape 클래스 정의
class VerticalLineDBShape(BaseShape):
    def __init__(self, center_time, strength_dB, thickness=1,
                 distribution_engine=None, distribution_type='none', distribution_params=None):
        super().__init__(distribution_engine, distribution_type, distribution_params)
        self.center_time = center_time
        self.strength_dB = strength_dB
        self.thickness = int(thickness)  # thickness를 정수로 변환

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        mask = np.zeros(spectro_shape)
        times = spectro_mod._get_times()
        time_idx = np.argmin(np.abs(times - self.center_time))
        start_idx = max(time_idx - self.thickness // 2, 0)
        end_idx = min(time_idx + self.thickness // 2 + 1, spectro_shape[1])
        mask[:, start_idx:end_idx] += self.strength_dB
        return mask

# 다른 Shape 클래스들 정의 (생략)

# DistributionEngine 클래스 정의 (예시로 간단히 구현)
class DistributionEngine:
    def normal(self, shape, mean=1, std=1):
        return np.random.normal(mean, std, shape)

# Pattern 클래스 정의 (생략)

# ShapeFactory 클래스 정의
class ShapeFactory:
    def __init__(self, distribution_engine):
        self.distribution_engine = distribution_engine

    def create(self, shape_name, **kwargs):
        if shape_name == "horizontal_line":
            return HorizontalLineDBShape(
                center_freq=kwargs['center_freq'],
                strength_dB=kwargs['strength_dB'],
                thickness=kwargs['thickness'],
                distribution_engine=self.distribution_engine,
                distribution_type=kwargs.get('distribution_type', 'none'),
                distribution_params=kwargs.get('distribution_params', None)
            )
        elif shape_name == "vertical_line":
            return VerticalLineDBShape(
                center_time=kwargs['center_time'],
                strength_dB=kwargs['strength_dB'],
                thickness=kwargs['thickness'],
                distribution_engine=self.distribution_engine,
                distribution_type=kwargs.get('distribution_type', 'none'),
                distribution_params=kwargs.get('distribution_params', None)
            )
        # 다른 shape_name에 대한 처리 추가 가능
        else:
            raise ValueError(f"Unknown shape name: {shape_name}")

# PatternFactory 클래스 정의 (생략)

# SpectrogramModifier 클래스 정의 (예시로 간단히 구현)
class SpectrogramModifier:
    def __init__(self, sample_rate, n_fft, hop_length, noise_strength=0, noise_type='none', noise_params=None):
        self.sample_rate = sample_rate
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.noise_strength = noise_strength
        self.noise_type = noise_type
        self.noise_params = noise_params
        self.S_db = None

    def compute_spectrogram(self, signal):
        S = np.abs(librosa.stft(signal, n_fft=self.n_fft, hop_length=self.hop_length))
        self.S_db = librosa.amplitude_to_db(S, ref=np.max)
        return self.S_db

    def apply_dB_mask(self, mask):
        self.S_db += mask

    def _get_freqs(self):
        return librosa.fft_frequencies(sr=self.sample_rate, n_fft=self.n_fft)

    def _get_times(self):
        return librosa.frames_to_time(np.arange(self.S_db.shape[1]), sr=self.sample_rate, hop_length=self.hop_length)

    def plot_spectrogram(self, show_labels=True, colormap='magma', title='Spectrogram'):
        plt.figure(figsize=(12, 6))
        librosa.display.specshow(self.S_db, sr=self.sample_rate, hop_length=self.hop_length, x_axis='time', y_axis='linear', cmap=colormap)
        plt.colorbar(format="%+2.0f dB")
        if show_labels:
            plt.title(title)
        plt.tight_layout()
        return plt.gcf(), plt.gca()

# NoisePipeline 클래스 정의
class NoisePipeline:
    def __init__(self, spectro_mod):
        self.spectro_mod = spectro_mod
        self.distribution_engine = DistributionEngine()
        self.shape_factory = ShapeFactory(self.distribution_engine)
        self.shapes = []
        self.patterns = []

    def add_shape(self, shape):
        self.shapes.append(shape)
        return self

    def add_pattern(self, pattern):
        if pattern is not None:
            self.patterns.append(pattern)
        return self

    def generate(self, signal):
        spec = self.spectro_mod.compute_spectrogram(signal)
        total_mask = np.zeros_like(spec)
        for shape in self.shapes:
            shape_mask = shape.create_mask(spec.shape, self.spectro_mod)
            total_mask += shape_mask
            # 각 shape의 마스크를 시각화 (디버깅용)
            shape.plot_mask(shape_mask)
        for pattern in self.patterns:
            pattern_mask = pattern.create_mask(spec.shape, self.spectro_mod, self.shape_factory, self.distribution_engine)
            total_mask += pattern_mask
            # 패턴의 마스크를 시각화 (디버깅용)
            pattern.plot_mask(pattern_mask)
        self.spectro_mod.apply_dB_mask(total_mask)
        return self.spectro_mod.S_db

# 랜덤 파라미터 생성 및 파이프라인 설정
shape_param_ranges = {
    "horizontal_line": {
        "center_freq": (0, 8000),
        "strength_dB": (10, 30),  # dB 강도 증가
        "thickness": (1, 50)
    },
    "vertical_line": {
        "center_time": (0, 12),
        "strength_dB": (10, 30),  # dB 강도 증가
        "thickness": (1, 50)
    }
    # 다른 shape에 대한 범위 설정 추가 가능
}

def random_param_value(range_tuple):
    return random.uniform(range_tuple[0], range_tuple[1])

def random_axis(axis_list):
    return random.choice(axis_list)

def random_vertices(num_points=5):
    return [(random_param_value((0, 8000)), random_param_value((0, 12))) for _ in range(num_points)]

def get_rotate_tuple(range_tuple):
    angle = random_param_value(range_tuple)
    return (angle, 0.0)  # 두 번째 값은 0.0 고정

shapes_to_add = ["horizontal_line", "vertical_line"]

random_shapes_params = []

for shape_name in shapes_to_add:
    params = {}
    ranges = shape_param_ranges[shape_name]

    for k, v in ranges.items():
        if k == "axis":
            params[k] = random_axis(v)
        elif k == "vertices":
            params[k] = random_vertices(random.randint(3, 6))
        elif k == "rotate":
            params[k] = get_rotate_tuple(v)
        elif k == "thickness":
            # thickness는 int로 변환
            params[k] = int(random_param_value(v))
        else:
            params[k] = random_param_value(v) if isinstance(v, tuple) else v

    try:
        shape_obj = ShapeFactory(DistributionEngine()).create(shape_name, **params)
        random_shapes_params.append({
            "shape_name": shape_name,
            **params,
            "distribution_type": 'none',
            "distribution_params": None
        })
    except Exception as e:
        print(f"Error creating shape '{shape_name}': {e}")

# 신호 생성: 여러 주파수의 사인파 합성
def generate_test_signal(sample_rate, duration):
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    frequencies = [440, 880, 1760]  # A4, A5, A6
    signal = np.zeros_like(t)
    for freq in frequencies:
        signal += 0.5 * np.sin(2 * np.pi * freq * t)
    # 화이트 노이즈 추가
    signal += 0.02 * np.random.normal(0, 1, len(t))  # 노이즈 강도 추가로 감소
    return signal

sample_rate = 16000
duration = 12  # seconds
signal = generate_test_signal(sample_rate, duration)

# SpectrogramModifier 설정
spectro_mod = SpectrogramModifier(
    sample_rate=sample_rate,
    n_fft=1024,
    hop_length=512,
    noise_strength=0,    # 기본 노이즈 제거
    noise_type='none',   # 노이즈 타입 'none'으로 설정
    noise_params=None    # 노이즈 파라미터 없음
)

# NoisePipeline에 SpectrogramModifier 전달
pipeline = NoisePipeline(spectro_mod)

# 파이프라인에 shapes 추가
for shape_params in random_shapes_params:
    shape_name = shape_params.pop('shape_name')
    shape_obj = ShapeFactory(DistributionEngine()).create(shape_name, **shape_params)
    pipeline.add_shape(shape_obj)

# 스펙트로그램 생성 및 마스크 적용
result_spectrogram = pipeline.generate(signal)

# 파라미터 JSON 출력
try:
    print(json.dumps(random_shapes_params, indent=4, ensure_ascii=False))
except TypeError as e:
    print(f"JSON serialization error: {e}")

# 이미지 표시
try:
    fig, ax = spectro_mod.plot_spectrogram(
        show_labels=True,
        colormap='magma',
        title='Random Shapes and Parameters Spectrogram'
    )
    plt.show()
except Exception as e:
    print(f"Error plotting spectrogram: {e}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
from abc import ABC, abstractmethod
import random
import json

# BaseShape 클래스 정의
class BaseShape(ABC):
    def __init__(self, distribution_engine=None, distribution_type='none', distribution_params=None):
        self.distribution_engine = distribution_engine
        self.distribution_type = distribution_type
        self.distribution_params = distribution_params

    @abstractmethod
    def generate_shape_mask(self, spectro_shape, spectro_mod):
        pass

    def create_mask(self, spectro_shape, spectro_mod):
        shape_mask = self.generate_shape_mask(spectro_shape, spectro_mod)
        # distribution_type에 따라 shape_mask 조정
        if self.distribution_type != 'none' and self.distribution_engine:
            if self.distribution_type == 'normal':
                mask_distribution = self.distribution_engine.normal(
                    shape_mask.shape, 
                    self.distribution_params.get('mean', 1), 
                    self.distribution_params.get('std', 1)
                )
                shape_mask *= mask_distribution
            # 다른 distribution_type에 대한 처리 추가 가능
        return shape_mask

    def plot_mask(self, mask):
        plt.figure(figsize=(10, 4))
        librosa.display.specshow(mask, sr=16000, hop_length=512, x_axis='time', y_axis='linear', cmap='cool')
        plt.colorbar(format="%+2.0f dB")
        plt.title(f"{self.__class__.__name__} Mask")
        plt.show()

# HorizontalLineDBShape 클래스 정의
class HorizontalLineDBShape(BaseShape):
    def __init__(self, center_freq, strength_dB, thickness=1,
                 distribution_engine=None, distribution_type='none', distribution_params=None):
        super().__init__(distribution_engine, distribution_type, distribution_params)
        self.center_freq = center_freq
        self.strength_dB = strength_dB
        self.thickness = int(thickness)  # thickness를 정수로 변환

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        mask = np.zeros(spectro_shape)
        freqs = spectro_mod._get_freqs()
        freq_idx = np.argmin(np.abs(freqs - self.center_freq))
        start_idx = max(freq_idx - self.thickness // 2, 0)
        end_idx = min(freq_idx + self.thickness // 2 + 1, spectro_shape[0])
        mask[start_idx:end_idx, :] += self.strength_dB
        return mask

# VerticalLineDBShape 클래스 정의
class VerticalLineDBShape(BaseShape):
    def __init__(self, center_time, strength_dB, thickness=1,
                 distribution_engine=None, distribution_type='none', distribution_params=None):
        super().__init__(distribution_engine, distribution_type, distribution_params)
        self.center_time = center_time
        self.strength_dB = strength_dB
        self.thickness = int(thickness)  # thickness를 정수로 변환

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        mask = np.zeros(spectro_shape)
        times = spectro_mod._get_times()
        time_idx = np.argmin(np.abs(times - self.center_time))
        start_idx = max(time_idx - self.thickness // 2, 0)
        end_idx = min(time_idx + self.thickness // 2 + 1, spectro_shape[1])
        mask[:, start_idx:end_idx] += self.strength_dB
        return mask

# CircleDBShape 클래스 정의
class CircleDBShape(BaseShape):
    def __init__(self, center_freq, center_time, radius_freq, radius_time, strength_dB,
                 distribution_engine=None, distribution_type='none', distribution_params=None):
        super().__init__(distribution_engine, distribution_type, distribution_params)
        self.center_freq = center_freq
        self.center_time = center_time
        self.radius_freq = radius_freq
        self.radius_time = radius_time
        self.strength_dB = strength_dB

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        mask = np.zeros(spectro_shape)
        freqs = spectro_mod._get_freqs()
        times = spectro_mod._get_times()
        center_freq_idx = np.argmin(np.abs(freqs - self.center_freq))
        center_time_idx = np.argmin(np.abs(times - self.center_time))

        for i in range(spectro_shape[0]):
            for j in range(spectro_shape[1]):
                if ((i - center_freq_idx) ** 2) / (self.radius_freq ** 2) + ((j - center_time_idx) ** 2) / (self.radius_time ** 2) <= 1:
                    mask[i, j] += self.strength_dB
        return mask

# RectangleDBShape 클래스 정의
class RectangleDBShape(BaseShape):
    def __init__(self, freq_min, freq_max, time_min, time_max, strength_dB,
                 distribution_engine=None, distribution_type='none', distribution_params=None):
        super().__init__(distribution_engine, distribution_type, distribution_params)
        self.freq_min = freq_min
        self.freq_max = freq_max
        self.time_min = time_min
        self.time_max = time_max
        self.strength_dB = strength_dB

    def generate_shape_mask(self, spectro_shape, spectro_mod):
        mask = np.zeros(spectro_shape)
        freqs = spectro_mod._get_freqs()
        times = spectro_mod._get_times()
        freq_min_idx = np.argmin(np.abs(freqs - self.freq_min))
        freq_max_idx = np.argmin(np.abs(freqs - self.freq_max))
        time_min_idx = np.argmin(np.abs(times - self.time_min))
        time_max_idx = np.argmin(np.abs(times - self.time_max))
        mask[freq_min_idx:freq_max_idx, time_min_idx:time_max_idx] += self.strength_dB
        return mask

# DistributionEngine 클래스 정의 (간단히 구현)
class DistributionEngine:
    def normal(self, shape, mean=1, std=1):
        return np.random.normal(mean, std, shape)

# ShapeFactory 클래스 정의
class ShapeFactory:
    def __init__(self, distribution_engine):
        self.distribution_engine = distribution_engine

    def create(self, shape_name, **kwargs):
        if shape_name == "horizontal_line":
            return HorizontalLineDBShape(
                center_freq=kwargs['center_freq'],
                strength_dB=kwargs['strength_dB'],
                thickness=kwargs['thickness'],
                distribution_engine=self.distribution_engine,
                distribution_type=kwargs.get('distribution_type', 'none'),
                distribution_params=kwargs.get('distribution_params', None)
            )
        elif shape_name == "vertical_line":
            return VerticalLineDBShape(
                center_time=kwargs['center_time'],
                strength_dB=kwargs['strength_dB'],
                thickness=kwargs['thickness'],
                distribution_engine=self.distribution_engine,
                distribution_type=kwargs.get('distribution_type', 'none'),
                distribution_params=kwargs.get('distribution_params', None)
            )
        elif shape_name == "circle":
            return CircleDBShape(
                center_freq=kwargs['center_freq'],
                center_time=kwargs['center_time'],
                radius_freq=kwargs['radius_freq'],
                radius_time=kwargs['radius_time'],
                strength_dB=kwargs['strength_dB'],
                distribution_engine=self.distribution_engine,
                distribution_type=kwargs.get('distribution_type', 'none'),
                distribution_params=kwargs.get('distribution_params', None)
            )
        elif shape_name == "rectangle":
            return RectangleDBShape(
                freq_min=kwargs['freq_min'],
                freq_max=kwargs['freq_max'],
                time_min=kwargs['time_min'],
                time_max=kwargs['time_max'],
                strength_dB=kwargs['strength_dB'],
                distribution_engine=self.distribution_engine,
                distribution_type=kwargs.get('distribution_type', 'none'),
                distribution_params=kwargs.get('distribution_params', None)
            )
        # 다른 shape_name에 대한 처리 추가 가능
        else:
            raise ValueError(f"Unknown shape name: {shape_name}")

# SpectrogramModifier 클래스 정의
class SpectrogramModifier:
    def __init__(self, sample_rate, n_fft, hop_length, noise_strength=0, noise_type='none', noise_params=None):
        self.sample_rate = sample_rate
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.noise_strength = noise_strength
        self.noise_type = noise_type
        self.noise_params = noise_params
        self.S_db = None

    def compute_spectrogram(self, signal):
        S = np.abs(librosa.stft(signal, n_fft=self.n_fft, hop_length=self.hop_length))
        self.S_db = librosa.amplitude_to_db(S, ref=np.max)
        return self.S_db

    def apply_dB_mask(self, mask):
        self.S_db += mask

    def _get_freqs(self):
        return librosa.fft_frequencies(sr=self.sample_rate, n_fft=self.n_fft)

    def _get_times(self):
        return librosa.frames_to_time(np.arange(self.S_db.shape[1]), sr=self.sample_rate, hop_length=self.hop_length)

    def plot_spectrogram(self, show_labels=True, colormap='magma', title='Spectrogram'):
        plt.figure(figsize=(12, 6))
        librosa.display.specshow(self.S_db, sr=self.sample_rate, hop_length=self.hop_length, x_axis='time', y_axis='linear', cmap=colormap)
        plt.colorbar(format="%+2.0f dB")
        if show_labels:
            plt.title(title)
        plt.tight_layout()
        return plt.gcf(), plt.gca()

# NoisePipeline 클래스 정의
class NoisePipeline:
    def __init__(self, spectro_mod):
        self.spectro_mod = spectro_mod
        self.distribution_engine = DistributionEngine()
        self.shape_factory = ShapeFactory(self.distribution_engine)
        self.shapes = []
        self.patterns = []

    def add_shape(self, shape):
        self.shapes.append(shape)
        return self

    def add_pattern(self, pattern):
        if pattern is not None:
            self.patterns.append(pattern)
        return self

    def generate(self, signal):
        spec = self.spectro_mod.compute_spectrogram(signal)
        total_mask = np.zeros_like(spec)
        for shape in self.shapes:
            shape_mask = shape.create_mask(spec.shape, self.spectro_mod)
            total_mask += shape_mask
            # 각 shape의 마스크를 시각화 (디버깅용)
            shape.plot_mask(shape_mask)
        for pattern in self.patterns:
            pattern_mask = pattern.create_mask(spec.shape, self.spectro_mod, self.shape_factory, self.distribution_engine)
            total_mask += pattern_mask
            # 패턴의 마스크를 시각화 (디버깅용)
            pattern.plot_mask(pattern_mask)
        self.spectro_mod.apply_dB_mask(total_mask)
        return self.spectro_mod.S_db

# 랜덤 파라미터 생성 및 파이프라인 설정
shape_param_ranges = {
    "horizontal_line": {
        "center_freq": (500, 7500),
        "strength_dB": (15, 30),  # dB 강도 증가
        "thickness": (5, 20)
    },
    "vertical_line": {
        "center_time": (1, 11),
        "strength_dB": (15, 30),  # dB 강도 증가
        "thickness": (5, 20)
    },
    "circle": {
        "center_freq": (1000, 7000),
        "center_time": (2, 10),
        "radius_freq": (200, 500),
        "radius_time": (0.5, 1.5),
        "strength_dB": (15, 30)
    },
    "rectangle": {
        "freq_min": (1000, 3000),
        "freq_max": (4000, 7000),
        "time_min": (2, 6),
        "time_max": (6, 10),
        "strength_dB": (15, 30)
    }
    # 필요한 다른 shape에 대한 범위 설정 추가 가능
}

def random_param_value(range_tuple):
    return random.uniform(range_tuple[0], range_tuple[1])

def random_vertices(num_points=5):
    return [(random_param_value((0, 8000)), random_param_value((0, 12))) for _ in range(num_points)]

def get_rotate_tuple(range_tuple):
    angle = random_param_value(range_tuple)
    return (angle, 0.0)  # 두 번째 값은 0.0 고정

shapes_to_add = ["horizontal_line", "vertical_line", "circle", "rectangle"]

random_shapes_params = []

for shape_name in shapes_to_add:
    params = {}
    ranges = shape_param_ranges[shape_name]

    for k, v in ranges.items():
        if k == "vertices":
            params[k] = random_vertices(random.randint(3, 6))
        elif k == "rotate":
            params[k] = get_rotate_tuple(v)
        elif k == "thickness":
            # thickness는 int로 변환
            params[k] = int(random_param_value(v))
        else:
            params[k] = random_param_value(v) if isinstance(v, tuple) else v

    try:
        shape_obj = ShapeFactory(DistributionEngine()).create(shape_name, **params)
        random_shapes_params.append({
            "shape_name": shape_name,
            **params,
            "distribution_type": 'none',
            "distribution_params": None
        })
    except Exception as e:
        print(f"Error creating shape '{shape_name}': {e}")

# 신호 생성: 여러 주파수의 사인파 합성
def generate_test_signal(sample_rate, duration):
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    frequencies = [440, 880, 1760]  # A4, A5, A6
    signal = np.zeros_like(t)
    for freq in frequencies:
        signal += 0.5 * np.sin(2 * np.pi * freq * t)
    # 화이트 노이즈 추가
    signal += 0.02 * np.random.normal(0, 1, len(t))  # 노이즈 강도 추가로 감소
    return signal

sample_rate = 16000
duration = 12  # seconds
signal = generate_test_signal(sample_rate, duration)

# SpectrogramModifier 설정
spectro_mod = SpectrogramModifier(
    sample_rate=sample_rate,
    n_fft=1024,
    hop_length=512,
    noise_strength=0,    # 기본 노이즈 제거
    noise_type='none',   # 노이즈 타입 'none'으로 설정
    noise_params=None    # 노이즈 파라미터 없음
)

# NoisePipeline에 SpectrogramModifier 전달
pipeline = NoisePipeline(spectro_mod)

# 파이프라인에 shapes 추가
for shape_params in random_shapes_params:
    shape_name = shape_params.pop('shape_name')
    shape_obj = ShapeFactory(DistributionEngine()).create(shape_name, **shape_params)
    pipeline.add_shape(shape_obj)

# 스펙트로그램 생성 및 마스크 적용
result_spectrogram = pipeline.generate(signal)

# 파라미터 JSON 출력
try:
    print(json.dumps(random_shapes_params, indent=4, ensure_ascii=False))
except TypeError as e:
    print(f"JSON serialization error: {e}")

# 이미지 표시
try:
    fig, ax = spectro_mod.plot_spectrogram(
        show_labels=True,
        colormap='magma',
        title='Random Shapes and Parameters Spectrogram'
    )
    plt.show()
except Exception as e:
    print(f"Error plotting spectrogram: {e}")
